## Hybrid Retriever- Combining Dense And Sparse Retriever

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

# EnsembleRetriever not in langchain_community; implement weighted RRF locally
class EnsembleRetriever(BaseRetriever):
    """Combine multiple retrievers with weighted Reciprocal Rank Fusion."""
    retrievers: list
    weights: list
    c: int = 60

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun = None):
        from langchain_core.documents import Document
        doc_sets = [r.invoke(query) for r in self.retrievers]
        fused = {}
        for docs, w in zip(doc_sets, self.weights):
            for rank, doc in enumerate(docs):
                key = (doc.page_content, tuple(doc.metadata.items()) if doc.metadata else ())
                fused[key] = fused.get(key, 0.0) + w / (self.c + rank + 1)
        sorted_docs = sorted(fused.items(), key=lambda x: -x[1])
        return [Document(page_content=c, metadata=dict(m)) for (c, m), _ in sorted_docs]



In [14]:
# Step 1: Sample documents
docs = [
    Document(page_content="LangChain helps build LLM applications."),
    Document(page_content="Pinecone is a vector database for semantic search."),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Langchain can be used to develop agentic ai application."),
    Document(page_content="Langchain has many types of retrievers.")
]

# Step 2: Dense Retriever (FAISS + HuggingFace)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

In [15]:
### Sparse Retriever(BM25)
sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=3 ##top- k documents to retriever

## step 4 : Combine with Ensemble Retriever
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weights=[0.7,0.3]
)


In [18]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x129b7c550>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x129b7d900>, k=3)], weights=[0.7, 0.3])

In [19]:
# Step 5: Query and get results
query = "How can I build an application using LLMs?"
results = hybrid_retriever.invoke(query)

# Step 6: Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
LangChain helps build LLM applications.

🔹 Document 2:
Langchain can be used to develop agentic ai application.

🔹 Document 3:
Langchain has many types of retrievers.

🔹 Document 4:
Pinecone is a vector database for semantic search.


### RAG Pipeline with hybrid retriever

In [20]:
from langchain.chat_models.base import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [21]:
# Step 5: Prompt Template (ChatPromptTemplate for chat models)
prompt = ChatPromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

## step 6-llm
llm = init_chat_model("openai:gpt-3.5-turbo", temperature=0.2)
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x129ae52a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12b06de10>, root_client=<openai.OpenAI object at 0x12a8c77f0>, root_async_client=<openai.AsyncOpenAI object at 0x12b06dd50>, temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [22]:
### RAG chain with LCEL (replaces create_stuff_documents_chain + create_retrieval_chain)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain returns {"context": docs, "answer": ...} to match legacy create_retrieval_chain output
rag_chain = (
    RunnablePassthrough.assign(context=lambda x: hybrid_retriever.invoke(x["input"]))
    | RunnablePassthrough.assign(
        answer=lambda x: (prompt | llm | StrOutputParser()).invoke(
            {"context": format_docs(x["context"]), "input": x["input"]}
        )
    )
)
rag_chain


RunnableAssign(mapper={
  context: RunnableLambda(lambda x: hybrid_retriever.invoke(x['input']))
})
| RunnableAssign(mapper={
    answer: RunnableLambda(lambda x: (prompt | llm | StrOutputParser()).invoke({'context': format_docs(x['context']), 'input': x['input']}))
  })

In [23]:
# Step 9: Ask a question
query = {"input": "How can I build an app using LLMs?"}
response = rag_chain.invoke(query)

# Step 10: Output
print("✅ Answer:\n", response["answer"])

print("\n📄 Source Documents:")
for i, doc in enumerate(response["context"]):
    print(f"\nDoc {i+1}: {doc.page_content}")

✅ Answer:
 You can build an app using LLMs by utilizing LangChain, which helps in developing LLM applications. LangChain can be used to create agentic AI applications and has various types of retrievers to enhance the functionality of your app. Additionally, you can also consider using Pinecone, a vector database for semantic search, to further improve the performance of your LLM-based app.

📄 Source Documents:

Doc 1: LangChain helps build LLM applications.

Doc 2: Langchain can be used to develop agentic ai application.

Doc 3: Langchain has many types of retrievers.

Doc 4: Pinecone is a vector database for semantic search.
